In [2]:
%load_ext autoreload  
%autoreload 2  
  
import random  
import numpy as np  
import torch  
import torch.nn as nn  
from torch.autograd import Function  
import optuna  
from modules.utils import evaluate_model  
import matplotlib.pyplot as plt  
from braindecode.models import EEGInceptionMI, EEGSimpleConv, MSVTNet, FBCNet, ATCNet
from braindecode import EEGClassifier  
import torch.optim as optim  
  
# dataset related  
from modules import CompetitionDataset, load_combined_moabb_data  
from torch.utils.data import DataLoader, TensorDataset  
from moabb.datasets import BNCI2014_001, PhysionetMI, Weibo2014, Cho2017  # 250 hz  
from braindecode.datasets import MOABBDataset, create_from_X_y  
from braindecode.preprocessing import preprocess, Preprocessor, exponential_moving_standardize, filterbank  
from skorch.helper import predefined_split  
from braindecode.datasets.base import BaseConcatDataset  
import random
import mne
import numpy as np
from skorch.callbacks import EpochScoring, Checkpoint, EarlyStopping, LRScheduler
from optuna.integration import SkorchPruningCallback


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  
device  

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


device(type='cuda')

In [3]:
# Add this at the beginning of your notebook, after imports
def set_random_seeds(seed=42):
    """Set random seeds for reproducibility"""

    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

# Call this function before creating datasets and models
set_random_seeds(42)

In [4]:
moabb_train_datasets = [
    PhysionetMI(imagined=True),  # 109 subjects
    Weibo2014(),  # 10 subjects, 64 channels
    CompetitionDataset(),
]
train_val = [CompetitionDataset(split="validation")]

# Load combined data # BEST tmin=1, tmax=4, resample=250
eeg_channels = ["Fz", "C3", "Cz", "C4", "Pz"]
X_train, y_train, domain_labels_train, info_train = load_combined_moabb_data(
    datasets=moabb_train_datasets,
    paradigm_config={
        "channels": eeg_channels,
        "tmin": 1.0,
        "tmax": 7.0,
        "resample": 250,
    }
    # subjects_per_dataset={
    #     "PhysionetMI": list(range(1, 2)),
    #     "Weibo2014": list(range(1, 2)),
    #     "CompetitionDataset": list(range(1, 2)),
    # },
)

# Load combined data
X_val, y_val, domain_labels_val, info_val = load_combined_moabb_data(
    datasets=train_val,
    paradigm_config={
        "channels": eeg_channels,
        "tmin": 1.0,
        "tmax": 7.0,
        "resample": 250,
    },
)

# Best preprocessor
# preprocessors = [
#     Preprocessor('resample', sfreq=250),
#     Preprocessor('filter', l_freq=1, h_freq=40),
#     # Preprocessor('pick_channels', ch_names=["C3", "Cz", "C4", "FC3", "FC4", "CP3", "CP4"]),
#     Preprocessor('set_eeg_reference', ref_channels='average'),  # Common average reference
#     Preprocessor(exponential_moving_standardize, factor_new=1e-3, init_block_size=1000),
#     Preprocessor(filterbank, frequency_bands=[(8, 12), (13, 30)], drop_original_signals=False),  # Mu/Beta bands
# ]


Processing dataset: PhysionetMI
Original subject range: 1 to 109
Used Annotations descriptions: ['left_hand', 'rest', 'right_hand']
Used Annotations descriptions: ['left_hand', 'rest', 'right_hand']
Used Annotations descriptions: ['left_hand', 'rest', 'right_hand']
Used Annotations descriptions: ['feet', 'hands', 'rest']
Used Annotations descriptions: ['feet', 'hands', 'rest']
Used Annotations descriptions: ['feet', 'hands', 'rest']
Used Annotations descriptions: ['left_hand', 'rest', 'right_hand']


c:\Users\ahmad\AppData\Local\Programs\Python\Python312\Lib\site-packages\moabb\datasets\preprocessing.py:278: UserWarning: warnEpochs <Epochs | 14 events (all good), 1 – 7 s (baseline off), ~559 KiB, data loaded,
 'left_hand': 7
 'right_hand': 7>
  warn(f"warnEpochs {epochs}")
c:\Users\ahmad\AppData\Local\Programs\Python\Python312\Lib\site-packages\moabb\datasets\preprocessing.py:278: UserWarning: warnEpochs <Epochs | 14 events (all good), 1 – 7 s (baseline off), ~559 KiB, data loaded,
 'left_hand': 7
 'right_hand': 7>
  warn(f"warnEpochs {epochs}")
c:\Users\ahmad\AppData\Local\Programs\Python\Python312\Lib\site-packages\moabb\datasets\preprocessing.py:278: UserWarning: warnEpochs <Epochs | 14 events (all good), 1 – 7 s (baseline off), ~559 KiB, data loaded,
 'left_hand': 7
 'right_hand': 7>
  warn(f"warnEpochs {epochs}")


Used Annotations descriptions: ['left_hand', 'rest', 'right_hand']
Used Annotations descriptions: ['left_hand', 'rest', 'right_hand']
Used Annotations descriptions: ['feet', 'hands', 'rest']
Used Annotations descriptions: ['feet', 'hands', 'rest']
Used Annotations descriptions: ['feet', 'hands', 'rest']
Used Annotations descriptions: ['left_hand', 'rest', 'right_hand']


c:\Users\ahmad\AppData\Local\Programs\Python\Python312\Lib\site-packages\moabb\datasets\preprocessing.py:278: UserWarning: warnEpochs <Epochs | 14 events (all good), 1 – 7 s (baseline off), ~559 KiB, data loaded,
 'left_hand': 7
 'right_hand': 7>
  warn(f"warnEpochs {epochs}")
c:\Users\ahmad\AppData\Local\Programs\Python\Python312\Lib\site-packages\moabb\datasets\preprocessing.py:278: UserWarning: warnEpochs <Epochs | 14 events (all good), 1 – 7 s (baseline off), ~559 KiB, data loaded,
 'left_hand': 7
 'right_hand': 7>
  warn(f"warnEpochs {epochs}")
c:\Users\ahmad\AppData\Local\Programs\Python\Python312\Lib\site-packages\moabb\datasets\preprocessing.py:278: UserWarning: warnEpochs <Epochs | 14 events (all good), 1 – 7 s (baseline off), ~559 KiB, data loaded,
 'left_hand': 7
 'right_hand': 7>
  warn(f"warnEpochs {epochs}")


Used Annotations descriptions: ['left_hand', 'rest', 'right_hand']
Used Annotations descriptions: ['left_hand', 'rest', 'right_hand']
Used Annotations descriptions: ['feet', 'hands', 'rest']
Used Annotations descriptions: ['feet', 'hands', 'rest']
Used Annotations descriptions: ['feet', 'hands', 'rest']
Used Annotations descriptions: ['left_hand', 'rest', 'right_hand']


...

Adjusted subject range: (110, 119)
Number of trials: 1580
Number of subjects: 10

Processing dataset: CompetitionDataset
Original subject range: 1 to 30


No stim channel nor annotations found, skipping setting annotations.
No stim channel nor annotations found, skipping setting annotations.
No stim channel nor annotations found, skipping setting annotations.
No stim channel nor annotations found, skipping setting annotations.
No stim channel nor annotations found, skipping setting annotations.
No stim channel nor annotations found, skipping setting annotations.
No stim channel nor annotations found, skipping setting annotations.
No stim channel nor annotations found, skipping setting annotations.
c:\Users\ahmad\AppData\Local\Programs\Python\Python312\Lib\site-packages\moabb\datasets\preprocessing.py:278: UserWarning: warnEpochs <Epochs | 10 events (all good), 1 – 7 s (baseline off), ~595 KiB, data loaded,
 'left_hand': 6
 'right_hand': 4>
  warn(f"warnEpochs {epochs}")
c:\Users\ahmad\AppData\Local\Programs\Python\Python312\Lib\site-packages\moabb\datasets\preprocessing.py:278: UserWarning: warnEpochs <Epochs | 10 events (all good), 1 – 

Adjusted subject range: (120, 149)
Number of trials: 2400
Number of subjects: 30

=== COMBINED DATASET SUMMARY ===
Total trials: 8571
Feature shape: (8571, 5, 1500)
Class distribution: [4273 4298]
Subject range: 1 to 149
Total unique subjects: 149

Processing dataset: CompetitionDataset
Original subject range: 1 to 30


No stim channel nor annotations found, skipping setting annotations.
c:\Users\ahmad\AppData\Local\Programs\Python\Python312\Lib\site-packages\moabb\datasets\preprocessing.py:278: UserWarning: warnEpochs <Epochs | 10 events (all good), 1 – 7 s (baseline off), ~595 KiB, data loaded,
 'left_hand': 7
 'right_hand': 3>
  warn(f"warnEpochs {epochs}")
No stim channel nor annotations found, skipping setting annotations.
c:\Users\ahmad\AppData\Local\Programs\Python\Python312\Lib\site-packages\moabb\datasets\preprocessing.py:278: UserWarning: warnEpochs <Epochs | 10 events (all good), 1 – 7 s (baseline off), ~595 KiB, data loaded,
 'left_hand': 5
 'right_hand': 5>
  warn(f"warnEpochs {epochs}")
No stim channel nor annotations found, skipping setting annotations.
c:\Users\ahmad\AppData\Local\Programs\Python\Python312\Lib\site-packages\moabb\datasets\preprocessing.py:278: UserWarning: warnEpochs <Epochs | 10 events (all good), 1 – 7 s (baseline off), ~595 KiB, data loaded,
 'left_hand': 6
 'right_




WARNING TASK DF EMPTY 6 AT ROW 36 AT SPLIT validation






WARNING TASK DF EMPTY 7 AT ROW 37 AT SPLIT validation






WARNING TASK DF EMPTY 8 AT ROW 38 AT SPLIT validation






WARNING TASK DF EMPTY 9 AT ROW 39 AT SPLIT validation






WARNING TASK DF EMPTY 10 AT ROW 40 AT SPLIT validation






WARNING TASK DF EMPTY 11 AT ROW 41 AT SPLIT validation






WARNING TASK DF EMPTY 12 AT ROW 42 AT SPLIT validation






WARNING TASK DF EMPTY 13 AT ROW 43 AT SPLIT validation






WARNING TASK DF EMPTY 14 AT ROW 44 AT SPLIT validation






WARNING TASK DF EMPTY 15 AT ROW 45 AT SPLIT validation






WARNING TASK DF EMPTY 16 AT ROW 46 AT SPLIT validation






WARNING TASK DF EMPTY 17 AT ROW 47 AT SPLIT validation






WARNING TASK DF EMPTY 18 AT ROW 48 AT SPLIT validation






WARNING TASK DF EMPTY 19 AT ROW 49 AT SPLIT validation






WARNING TASK DF EMPTY 20 AT ROW 50 AT SPLIT validation






WARNING TASK DF EMPTY 21 AT ROW 51 AT SPLIT validation






WARNING TASK DF EMPTY 22 

In [4]:
def preprocess_numpy(
    X,  # shape (n_trials, n_chans, n_times)
    sfreq=250,
    l_freq=1.0,
    h_freq=40.0,
    ref="average",
    bands=[(8, 12), (13, 30)],
    ems_alpha=1e-3,
    ems_init_block=1000,
    ems_eps=1e-4,
):
    # a) Bandpass filter the full signal
    X_filt = np.stack([mne.filter.filter_data(trial, sfreq, l_freq, h_freq, verbose=False) for trial in X], axis=0)

    # b) Common‑average reference
    if ref == "average":
        X_filt = X_filt - X_filt.mean(axis=1, keepdims=True)
    else:
        X_filt = X_filt

    # c) Exponential‑moving standardize per trial & channel
    #    m_t and v_t initialized from first ems_init_block samples
    n_trials, n_chans, n_times = X_filt.shape
    X_filt = np.zeros_like(X_filt)
    for i in range(n_trials):
        trial = X_filt[i]
        # initialize m, v
        m = trial[:, :ems_init_block].mean(axis=1, keepdims=True)  # (n_chans,1)
        v = trial[:, :ems_init_block].var(axis=1, keepdims=True)  # (n_chans,1)
        for t in range(n_times):
            x_t = trial[:, t : t + 1]  # (n_chans,1)
            # update
            m = (1 - ems_alpha) * m + ems_alpha * x_t
            v = (1 - ems_alpha) * v + ems_alpha * ((x_t - m) ** 2)
            # standardize
            X_filt[i, :, t] = ((x_t - m) / np.sqrt(v + ems_eps)).ravel()

    # d) Filterbank: for each band, bandpass the EMS‑normalized signals
    # fb_list = []
    # for low, high in bands:
    #     fb = np.stack([mne.filter.filter_data(trial, sfreq, low, high, verbose=False) for trial in X_filt], axis=0)
    #     fb_list.append(fb)
    # X_filt = np.concatenate(fb_list, axis=1)  # now shape (n_trials, n_chans*len(bands), n_times)

    return X_filt


# for some reason i can't understand preprocessing kills model accuracy

# X_train = preprocess_numpy(
#     X_train,
#     sfreq=250,
#     l_freq=1,
#     h_freq=40,
#     ref="average",
#     bands=[(8, 12), (13, 30)],
#     ems_alpha=1e-3,
#     ems_init_block=1000,
# )
# X_val = preprocess_numpy(
#     X_val,
#     sfreq=250,
#     l_freq=1,
#     h_freq=40,
#     ref="average",
#     bands=[(8, 12), (13, 30)],
#     ems_alpha=1e-3,
#     ems_init_block=1000,
# )

In [5]:
X_train = torch.from_numpy(X_train).float()  # FloatTensor of shape (N, C, T)
y_train = torch.from_numpy(y_train).long()  # LongTensor of shape (N, 2)

train_dataset = TensorDataset(X_train, y_train)

X_val_t = torch.from_numpy(X_val).float()
y_val_t = torch.from_numpy(y_val).long()

val_dataset = TensorDataset(X_val_t, y_val_t)
X_train.shape

torch.Size([8571, 5, 1500])

In [ ]:
def set_seed(seed):  
    torch.manual_seed(seed)  
    torch.cuda.manual_seed_all(seed)  
    torch.backends.cudnn.deterministic = True  
    torch.backends.cudnn.benchmark = False  
  
def objective(trial):    
    set_seed(42 + trial.number)    
    
    # EEGSimpleConv hyperparameters - based on paper recommendations    
    feature_maps = trial.suggest_int("feature_maps", 64, 144, step=16)    
    n_convs = trial.suggest_int("n_convs", 2, 4)    
    kernel_size = trial.suggest_int("kernel_size", 5, 8)    
        
    # IMPORTANT: Tune resampling frequency - this is critical for performance    
    resampling_freq = trial.suggest_int("resampling_freq", 50, 100, step=10)    
        
    # Activation function tuning    
    activation_name = trial.suggest_categorical("activation", ["ReLU", "ELU", "LeakyReLU", "GELU"])    
    activation_map = {    
        "ReLU": torch.nn.ReLU,    
        "ELU": torch.nn.ELU,     
        "LeakyReLU": torch.nn.LeakyReLU,    
        "GELU": torch.nn.GELU    
    }    
    activation = activation_map[activation_name]    
    
    # Training hyperparameters    
    lr = trial.suggest_float("lr", 1e-5, 1e-2, log=True)  # Expanded range    
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64, 128, 256])    
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-2, log=True)    
        
    # Scheduler parameters    
    use_scheduler = trial.suggest_categorical("use_scheduler", [True, False])    
    if use_scheduler:    
        scheduler_step_size = trial.suggest_int("scheduler_step_size", 10, 100, step=10)    
        scheduler_gamma = trial.suggest_float("scheduler_gamma", 0.1, 0.9, step=0.1)    
        
    # Early stopping patience    
    patience = trial.suggest_int("patience", 10, 50, step=5)    
        
    n_chans = X_train.shape[1]    
    n_outputs = y_train.unique().numel()    
    
    # Initialize model with tuned parameters    
    model = EEGSimpleConv(    
        n_chans=n_chans,    
        n_outputs=n_outputs,    
        sfreq=250,  # Your original data frequency    
        feature_maps=feature_maps,    
        n_convs=n_convs,    
        kernel_size=kernel_size,    
        resampling_freq=resampling_freq,  # Now tuned!    
        activation=activation,  # Now tuned!    
    )    
    
    import os    
    os.makedirs('./checkpoints/mi/optuna', exist_ok=True)    
    
    callbacks = [    
        "accuracy",    
        SkorchPruningCallback(trial, monitor='valid_accuracy'),    
        Checkpoint(    
            f_params=f'./checkpoints/mi/optuna/best_model_trial_{trial.number}.pth',    
            monitor='valid_accuracy',    
            load_best=True,    
        ),    
        EarlyStopping(monitor='valid_accuracy', patience=patience, lower_is_better=False),    
    ]    
        
    # Setup scheduler if enabled    
    if use_scheduler:    
        callbacks.append(    
            LRScheduler(    
                policy='StepLR',    
                step_size=scheduler_step_size,    
                gamma=scheduler_gamma    
            )    
        )    
    
    clf = EEGClassifier(    
        model,    
        criterion=torch.nn.CrossEntropyLoss,    
        optimizer=torch.optim.Adam,    
        optimizer__lr=lr,    
        optimizer__weight_decay=weight_decay,    
        batch_size=batch_size,    
        max_epochs=200,    
        train_split=predefined_split(val_dataset),    
        device="cuda" if torch.cuda.is_available() else "cpu",    
        verbose=0,    
        callbacks=callbacks,    
    )    
    
    clf.fit(train_dataset)    
    return clf.history[-1, 'valid_accuracy']    
  
# Create an Optuna study and run the optimization  
study = optuna.create_study(  
    direction="maximize",  
    pruner=optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=10, interval_steps=1)  
)  
  
# Run the optimization for a certain number of trials  
study.optimize(objective, n_trials=50) # Adjust n_trials as needed  
  
print("\nOptimization finished.")  
print("Best trial:")  
trial = study.best_trial  
  
print(f"  Value: {trial.value}")  
print("  Params: ")  
for key, value in trial.params.items():  
    print(f"    {key}: {value}")  
  
# --- Load the best model after optimization ---  
best_params = study.best_trial.params  
n_chans = X_train.shape[1]  
n_outputs = y_train.unique().numel()  
  
activation_map = {    
    "ReLU": torch.nn.ReLU,    
    "ELU": torch.nn.ELU,     
    "LeakyReLU": torch.nn.LeakyReLU,    
    "GELU": torch.nn.GELU    
}    
    
final_model = EEGSimpleConv(    
    n_chans=n_chans,    
    n_outputs=n_outputs,    
    sfreq=250,  # Your original data frequency    
    feature_maps=best_params["feature_maps"],    
    n_convs=best_params["n_convs"],    
    kernel_size=best_params["kernel_size"],    
    resampling_freq=best_params["resampling_freq"],  # Now included!    
    activation=activation_map[best_params["activation"]],  # Now included!    
)  
  
# Reinitialize the classifier with the best parameters  
final_clf = EEGClassifier(  
    final_model,  
    criterion=torch.nn.CrossEntropyLoss,  
    optimizer=torch.optim.Adam,  
    optimizer__lr=best_params["lr"],  
    optimizer__weight_decay=best_params["weight_decay"],  # Added weight_decay  
    batch_size=best_params["batch_size"],  
    max_epochs=1000, # Or a reduced number for a final training run  
    train_split=predefined_split(val_dataset), # You might want to combine train+val here for final training  
    device="cuda" if torch.cuda.is_available() else "cpu",  
    verbose=1,  
    callbacks=["accuracy"],  
)  
  
# Load the state_dict from the checkpoint of the best trial  
best_model_path = f'./checkpoints/mi/optuna/best_model_trial_{study.best_trial.number}.pth'    
final_clf.initialize() # Initialize the model structure    
final_clf.module_.load_state_dict(torch.load(best_model_path))    
  
print(f"Loaded best model from trial {study.best_trial.number} for final use from: {best_model_path}")  
  
# Optionally, save this best model as your final optimized model  
torch.save(final_clf.module_.state_dict(), './checkpoints/mi/eegsimpleconv_optimized_final.pth')


[I 2025-06-30 15:12:11,618] A new study created in memory with name: no-name-31c0ccd8-f2cb-43f9-8f0e-0cffb20a60db
c:\Users\ahmad\AppData\Local\Programs\Python\Python312\Lib\site-packages\skorch\callbacks\training.py:214: UserWarning: Checkpoint monitor parameter is set to 'valid_accuracy' and the history contains 'valid_accuracy_best'. Perhaps you meant to set the parameter to 'valid_accuracy_best'
  warnings.warn(
c:\Users\ahmad\AppData\Local\Programs\Python\Python312\Lib\site-packages\skorch\callbacks\training.py:214: UserWarning: Checkpoint monitor parameter is set to 'valid_accuracy' and the history contains 'valid_accuracy_best'. Perhaps you meant to set the parameter to 'valid_accuracy_best'
  warnings.warn(
c:\Users\ahmad\AppData\Local\Programs\Python\Python312\Lib\site-packages\skorch\callbacks\training.py:214: UserWarning: Checkpoint monitor parameter is set to 'valid_accuracy' and the history contains 'valid_accuracy_best'. Perhaps you meant to set the parameter to 'valid_acc

In [23]:
# [I 2025-06-30 18:30:33,124] Trial 23 finished with value: 0.64 and parameters: {'feature_maps': 112, 'n_convs': 2, 'kernel_size': 8, 'resampling_freq': 100, 'activation': 'ELU', 'lr': 2.13537735805541e-05, 'batch_size': 128, 'weight_decay': 0.0002624302515969059, 'use_scheduler': False, 'patience': 40}. Best is trial 23 with value: 0.64.
n_chans = 5
n_outputs = 2
n_convs = 2
kernel_size = 8
feature_maps = 112
resampling_freq = 100
activation = torch.nn.ELU

model = EEGSimpleConv(    
        n_chans=n_chans,    
        n_outputs=n_outputs,    
        sfreq=250,
        feature_maps=feature_maps,    
        n_convs=n_convs,    
        kernel_size=kernel_size,    
        resampling_freq=resampling_freq, 
        activation=activation, 
)    
    
clf = EEGClassifier(  
    model,  
    criterion=torch.nn.CrossEntropyLoss,  
    optimizer=torch.optim.Adam,  
    optimizer__lr=2.13537735805541e-05,  
    optimizer__weight_decay=0.0002624302515969059,  
    batch_size=128,  
    max_epochs=20, # Or a reduced number for a final training run  
    train_split=predefined_split(val_dataset), # You might want to combine train+val here for final training  
    device="cuda" if torch.cuda.is_available() else "cpu",  
    verbose=1,  
    callbacks=["accuracy"],  
)  

try:
    clf.initialize()  
    clf.module_.load_state_dict(torch.load('./checkpoints/mi/eegsimpleconv.pth'))
except Exception:
    print('no module to load')
    
clf.fit(X_train, y_train)
torch.save(clf.module_.state_dict(), './checkpoints/mi/eegsimpleconv.pth')

Re-initializing module.
Re-initializing criterion.
Re-initializing optimizer.
  epoch    train_accuracy    train_loss    valid_acc    valid_accuracy    valid_loss      dur
-------  ----------------  ------------  -----------  ----------------  ------------  -------
      1            0.9698        0.0481       0.6200            0.6200        2.0302  13.3856
      2            0.9438        0.0398       0.5000            0.5000        2.9970  13.3828
      3            0.9894        0.0351       0.5200            0.5200        2.5407  13.5859
      4            0.9995        0.0335       0.5000            0.5000        2.8832  13.0484
      5            0.9998        0.0349       0.5000            0.5000        2.4805  12.6960
      6            0.9935        0.0432       0.4800            0.4800        2.7160  13.4868
      7            0.9644        0.0761       0.5200            0.5200        1.7965  13.5947
      8            0.9435        0.0423       0.5800            0.5800      

In [20]:
torch.save(clf.module_.state_dict(), './checkpoints/mi/eegsimpleconv.pth')